# Network Data Cleaning
The goal of this notebook is to clean up the invalid data we have from weird timestamps

In [1]:
import json
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import display, clear_output
import csv as csv
import os
import subprocess
from tqdm import tqdm
import matplotlib.pyplot as mpl
import matplotlib.cm as cm
import matplotlib
import pylab as pl

In [2]:
pwd = "/home/srivbane/shared/caringbridge/data/projects/sna-social-support/csv_data/"
with open(os.path.join(pwd, "dynamic_ints.csv"), 'r', encoding="utf-8") as ints:
    with open (os.path.join(pwd, "dynamic_auth.csv"), 'r', encoding="utf-8") as auths:
        a = pd.read_csv(auths, index_col = 0, header=None, names=("total", "first", "last"))
        i = pd.read_csv(ints, index_col = 0, header=None, names=("total", "first", "last"))

### Time Traveling?
Looks like bad data!

In [3]:
tt = []
count = 0
num_years = 10
epoch_yr = 31557600000 # accounting for milliseconds
for index, auth in a.iterrows():
    try:
        user = (i.loc[index, :])
        if np.abs(auth["first"] - user["first"]) > (num_years * epoch_yr) or \
        np.abs(auth["last"] - user["last"]) > (num_years * epoch_yr):
            problem = [index, auth["first"], auth["last"], user["first"], user["last"]]
            tt.append(problem)
    except KeyError:
        pass

In [50]:
tu = []
count = 0
num_years = 10
epoch_yr = 31557600000 # accounting for milliseconds
for index, auth in i.iterrows():
    try:
        user = (a.loc[index, :])
        if np.abs(auth["first"] - user["first"]) > (num_years * epoch_yr) or \
        np.abs(auth["last"] - user["last"]) > (num_years * epoch_yr):
            problem = [index, auth["first"], auth["last"], user["first"], user["last"]]
            tt.append(problem)
    except KeyError:
        pass

In [6]:
df_a = pd.DataFrame(tt, columns = ("Uid", "First Auth", "Last Auth", "First Int", "Last Int"))
#df_b = pd.DataFrame(tu, columns = ("Uid", "First Auth", "Last Auth", "First Int", "Last Int"))

### Filtering the bad data out
Ran this in a script on 3/8 on mesabi, all done

In [ ]:
filter_me = []
count = 0
for index, i_ in df_a.iterrows():
    for val in i_:
        if val == i_[0]:
            continue;
        try:
            # get rid of > 2019, < 1997, or peculiar singles 
            if (val / epoch_yr) > 49 or (val / epoch_yr) < 27 or (i_[1] == i_[2]):
                if not i_[0] in filter_me:
                    filter_me.append(i_[0])
                    count += 1
        except:
            continue;
for index, i_ in df_b.iterrows():
    for val in i_:
        if val == i_[0]:
            continue;
        try:
            if (val / epoch_yr) > 49 or (val / epoch_yr) < 27 or (i_[1] == i_[2]):
                if not i_[0] in filter_me:
                    filter_me.append(i_[0])
                    count += 1
        except:
            continue;
display("Preparing to purge " + str(count) + " users from the dynamic data.");

'Preparing to purge 221 users from the dynamic data.'

In [ ]:
timer = 0
with open (os.path.join(pwd, "dynamic_ints.csv"), 'r', encoding="utf-8") as ints, \
open (os.path.join(pwd, "dynamic_auth.csv"), 'r', encoding="utf-8") as auths, \
open (os.path.join(pwd, "f_dynamic_ints.csv"), 'w', encoding="utf-8") as f_ints, \
open (os.path.join(pwd, "f_dynamic_auth.csv"), 'w', encoding="utf-8") as f_auths:
    ints_r = csv.reader(ints); auths_r = csv.reader(auths); 
    ints_w = csv.writer(f_ints); auths_w = csv.writer(f_auths);
    for row in ints_r:
        if(int(row[0]) not in filter_me):
            ints_w.writerow(row);
            timer += 1;
            if timer % 100000 == 1:
                display(timer)
    display("Almost there.");
    for row in auths_r:
        if(int(row[0]) not in filter_me):
            auths_w.writerow(row);
            timer += 1;
            if timer % 100000 == 1:
                display(timer)
display("Finished.")